In [3]:
import pandas as pd
import numpy as np
from skopt import BayesSearchCV #scikit-optimize
from skopt.space import Real, Categorical, Integer
from xgboost import XGBClassifier


ModuleNotFoundError: No module named 'importlib.metadata'

In [2]:

#!pip install scikit-optimize
#!pip install xgboost
#!pip install importlib
#!pip install importlib-metadata
#!pip install importlib_metadata

In [73]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [74]:
def process_df(df,supp=None):
    class data():
        def __init__(self):
            self.df=dict()

        def add(self,x):
            self.df[x[0]]=x[1]
    
    df.columns=[col.replace(' ','_') for col in df.columns]
    
    df['have_job']=(~df['Profession'].isna())*1
    df['Profession']=df['Profession'].fillna('isnone')
    df['Degree']=df['Degree'].fillna('isnone')
    
    df['Pressure']=df['Work_Pressure']
    df['Pressure']=df['Pressure'].fillna(df['Academic_Pressure'])
    df=df.drop(columns=['Work_Pressure','Academic_Pressure'])
    
    
    df['Satisfaction']=df['Job_Satisfaction']
    df['Satisfaction']=df['Satisfaction'].fillna(df['Study_Satisfaction'])
    df=df.drop(columns=['Job_Satisfaction','Study_Satisfaction'])
    
    df['Pressure']=df['Pressure'].fillna(df['Pressure'].median())
    df['Satisfaction']=df['Satisfaction'].fillna(df['Satisfaction'].median())
    df['Financial_Stress']=df['Financial_Stress'].fillna(df['Financial_Stress'].median())
    
    df=df.drop(columns='CGPA')
    
    rank=['Dietary_Habits','City','Sleep_Duration','Age','Name','Profession','Degree','Work/Study_Hours','Financial_Stress']
    rank_and_orig=['Age','Work/Study Hours','Financial_Stress']
    
    if supp is None:
        supp=data()
        #good_dietary_habits=pd.DataFrame(train['Dietary Habits'].value_counts()).head(3).index
        #supp.add(['good_dietary_habits',good_dietary_habits])
        for col in rank:

            temp=pd.DataFrame(df[col].value_counts()).reset_index()
            temp.columns=[col,'cnt']
            temp['cut']=temp['cnt'].cumsum()/temp['cnt'].sum()
            temp[col+'_rank']=(temp['cut']//0.2).astype(int)
            temp['last_cnt']=temp.cnt.shift(1).fillna(0)
            temp['escalation']=temp.last_cnt/temp.cnt

            for i,row in temp.iterrows():
                if row.escalation>11:
                    temp.loc[temp.index>=i,col+'_rank']=temp.loc[temp.index>=i,col+'_rank']+1

            print(temp.groupby(col+'_rank',as_index=False).agg({'cnt':['max','min','count']}))

            temp=temp[[col,col+'_rank']]
            supp.add([col.replace(' ','_'),temp])
        

    #df.loc[~df['Dietary Habits'].isin(good_dietary_habits),'Dietary Habits']='Other'
    #temp=pd.get_dummies(df['Dietary Habits'],drop_first=True)
    #df=pd.concat([df,temp],axis=1)
    #df=df.drop(columns='Dietary Habits')
    
    for col in rank:
        df=df.merge(supp.df[col],how='left',on=col)
        df[col+'_rank']=df[col+'_rank'].fillna(df[col+'_rank'].max()) # если нет в трейне, но есть в тесте - относим к редким
        if col not in rank_and_orig:
            df=df.drop(columns=col)
    
    #
    for col in ([col+'_rank' for col in rank]+['Gender','Have_you_ever_had_suicidal_thoughts_?','Family_History_of_Mental_Illness','Working_Professional_or_Student']):
        temp=pd.get_dummies(df[col],drop_first=True,prefix='_'.join(col.split('_')[-3:]))
        df=pd.concat([df,temp],axis=1)
        df=df.drop(columns=col)
    return df,supp

In [75]:
train,supp=process_df(train)
test,supp=process_df(test,supp)

  Dietary_Habits_rank    cnt             
                         max    min count
0                   1  49705  49705     1
1                   3  46227  46227     1
2                   4  44741  44741     1
3                   5      2      1    20
  City_rank   cnt            
              max   min count
0         0  6591  5689     4
1         1  5613  5176     6
2         2  5074  4636     5
3         3  4606  4280     7
4         4  4209  3144     8
5         5     7     1    68
  Sleep_Duration_rank    cnt             
                         max    min count
0                   1  38784  38784     1
1                   2  36969  36969     1
2                   3  32726  32726     1
3                   4  32142  32142     1
4                   5     12      1    32
  Age_rank   cnt            
             max   min count
0        0  5246  4199     6
1        1  4080  3695     7
2        2  3535  3217     8
3        3  3140  2742    10
4        4  2724  1785    12
  Name_rank

In [77]:
test.shape

(93800, 46)

In [60]:
train.isna().sum()

id                                              0
Age                                             0
Financial_Stress                                0
Depression                                      0
have_job                                        0
Pressure                                        0
Satisfaction                                    0
Name_rank                                       0
Profession_rank                                 0
Degree_rank                                     0
Work/Study_Hours_rank                           0
Financial_Stress_rank                           0
Sleep_Duration_rank_2                           0
Sleep_Duration_rank_3                           0
Sleep_Duration_rank_4                           0
Sleep_Duration_rank_5                           0
Dietary_Habits_rank_3.0                         0
Dietary_Habits_rank_4.0                         0
Dietary_Habits_rank_5.0                         0
City_rank_1                                     0


In [134]:

temp.head(40)

,City,cnt,cut,City_rank,last_cnt,delta
0,Kalyan,6591,0.046844,0.0,0.0,0.000000
1,Patna,5924,0.088948,0.0,6591.0,1.112593
2,Vasai-Virar,5765,0.129922,0.0,5924.0,1.027580
3,Kolkata,5689,0.170355,0.0,5765.0,1.013359
4,Ahmedabad,5613,0.210249,1.0,5689.0,1.013540
5,Meerut,5528,0.249538,1.0,5613.0,1.015376
6,Ludhiana,5226,0.286681,1.0,5528.0,1.057788
7,Pune,5210,0.323710,1.0,5226.0,1.003071
8,Rajkot,5207,0.360718,1.0,5210.0,1.000576
9,Visakhapatnam,5176,0.397505,1.0,5207.0,1.005989


In [132]:
temp.head(40)

,City,cnt,cut,City_rank,last_cnt,delta
0,Kalyan,6591,0.046844,0.0,0.0,0.000000
1,Patna,5924,0.088948,0.0,6591.0,1.112593
2,Vasai-Virar,5765,0.129922,1.0,5924.0,1.027580
3,Kolkata,5689,0.170355,1.0,5765.0,1.013359
4,Ahmedabad,5613,0.210249,2.0,5689.0,1.013540
5,Meerut,5528,0.249538,2.0,5613.0,1.015376
6,Ludhiana,5226,0.286681,2.0,5528.0,1.057788
7,Pune,5210,0.323710,3.0,5226.0,1.003071
8,Rajkot,5207,0.360718,3.0,5210.0,1.000576
9,Visakhapatnam,5176,0.397505,3.0,5207.0,1.005989
